## Adding neighbourhood name to main df

In [2]:
# Importing libraries
import geopandas as gpd
import pandas as pd

# Opening datasets
neighbourhoods = gpd.read_file('Data/limites-administratives-des-communes-en-region-de-bruxelles-capitale.geojson')
listings = pd.read_excel('Data/listings.xlsx')

In [3]:
neighbourhoods.columns.tolist()

['ogc_fid',
 'inspire_id',
 'national_code',
 'urbis_id',
 'name_fr',
 'name_nl',
 'versionid',
 're_id',
 'geo_point_2d',
 'geometry']

In [4]:
# Your existing code
airbnb_geo = gpd.GeoDataFrame(
    listings,
    geometry=gpd.points_from_xy(listings['longitude'], listings['latitude']),
    crs='EPSG:4326'
)

# Make sure coordinate systems match
neighbourhoods = neighbourhoods.to_crs('EPSG:4326')

# Spatial join - this adds the NAME_FR column to your airbnb data
airbnb_with_neighbourhood = gpd.sjoin(airbnb_geo, neighbourhoods[['name_fr', 'geometry']], 
                               how='left', 
                               predicate='within')

# Getting rid of geometry column
airbnb_with_neighbourhood = airbnb_with_neighbourhood.drop(columns=['geometry'])
airbnb_with_neighbourhood = airbnb_with_neighbourhood.drop(columns=['index_right'])

In [5]:
airbnb_with_neighbourhood.to_csv('Data/listings_final.csv',
                                 index = False,
                                 encoding = 'utf-8-sig',  # Works in both Excel and Tableau
                                 sep=',',
                                 decimal='.',
                                 date_format='%Y-%m-%d',
                                 float_format='%.6f')